In [65]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import time
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from scipy import stats
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier
#from bert_serving.client import BertClient
from sklearn.multiclass import OneVsOneClassifier
#import gensim
#from gensim.utils import simple_preprocess
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
#import gensim.corpora as corpora
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.sparse import hstack
#import gensim.downloader as api
#from gensim.corpora import Dictionary
#from gensim.similarities import SparseTermSimilarityMatrix
#from gensim.similarities import WordEmbeddingSimilarityIndex
#from gensim.models import TfidfModel
#from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

In [66]:
import warnings
warnings.filterwarnings('ignore')

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def vectorization_TF_IDF(DATASET, text_field_name, label_name):

    X_train, X_test, y_train, y_test = train_test_split( DATASET[text_field_name] ,
                                                    DATASET[label_name],
                                                    train_size=0.80, random_state=28)

    encoder = preprocessing.LabelEncoder()
    y_train = encoder.fit_transform(y_train)
    y_test = encoder.fit_transform(y_test)

    return X_train, X_test, y_train, y_test

def saving_results(results, path, file_name):

    results.to_csv(path + file_name,  index = False, sep=";")

def train_model(classifier, X_train_v, X_test_v, y_train, y_test):
    ini = time.time()
    classifier.fit(X_train_v, y_train)
    predictions = classifier.predict(X_test_v)
    fim = time.time()

    timee = fim-ini
    acuracia = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    matriz_confusao =  confusion_matrix(y_test, predictions)
    recall = recall_score(y_test, predictions, average='weighted')
    f1_score = metrics.f1_score(y_test, predictions, average='weighted')
    print("Time: " + str(timee))
    print("Acurácia: " + str(acuracia))
    print("Precision: " + str(precision))
    print("Recal: " + str(recall))
    print("f1_score: " + str(f1_score))

    return timee , acuracia, precision, recall, f1_score, matriz_confusao

def get_tests_result(X_train_v, X_test_v, y_train, y_test):

    all_res = []
    ### MLPClassifier
    print( "MLPClassifier")
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(70, ), random_state=1, verbose=True)
    timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)
    print(matriz_confusao)
    all_res.append(["MLPClassifier: ", timee , acuracia, precision, recall, f1_score])


    ### AdaBoostClassifier
    print( "AdaBoostClassifier: ")
    clf = AdaBoostClassifier(n_estimators= 50, learning_rate=1)
    timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)
    print(matriz_confusao)
    all_res.append(["AdaBoostClassifier: ", timee , acuracia, precision, recall, f1_score])

    ### Voting_LR3_SVC1_ETC2
    print( "Voting_LR3_SVC1_ETC2: ")
    ini = time.time()
    clf2 = LogisticRegression(random_state=0)
    clf5 = SVC(kernel='rbf', probability=True)
    clf6 = ExtraTreesClassifier(n_estimators=100, random_state=0)
    eclf = VotingClassifier(estimators=[ ('lr', clf2), ('svc', clf5), ('etc', clf6)], voting='soft', weights=[3, 1, 2])
    clf2 = clf2.fit(X_train_v, y_train)
    clf5 = clf5.fit(X_train_v, y_train)
    clf6 = clf6.fit(X_train_v, y_train)
    eclf = eclf.fit(X_train_v, y_train)
    #Y_previsto_vc1 = eclf.predict(X_test_v.toarray())
    Y_previsto_vc1 = eclf.predict(X_test_v)
    fim = time.time()

    timee = fim-ini
    acuracia = accuracy_score(y_test, Y_previsto_vc1)
    precision = precision_score(y_test, Y_previsto_vc1, average='weighted')
    matriz_confusao =  confusion_matrix(y_test, Y_previsto_vc1)
    recall = recall_score(y_test, Y_previsto_vc1, average='weighted')
    f1_score = metrics.f1_score(y_test, Y_previsto_vc1, average='weighted')
    print("Time: " + str(timee))
    print("Acurácia: " + str(acuracia))
    print("Precision: " + str(precision))
    print("Recal: " + str(recall))
    print("f1_score: " + str(f1_score))
    print(matriz_confusao)
    all_res.append(["Voting_LR3_SVC1_ETC2: ", timee , acuracia, precision, recall, f1_score])

    ### DecisionTreeClassifier
    print( "DecisionTreeClassifier: ")
    clf = DecisionTreeClassifier(random_state=0)
    timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)
    print(matriz_confusao)
    all_res.append(["DecisionTreeClassifier: ", timee , acuracia, precision, recall, f1_score])

    ### RandomForestClassifier
    print( "RandomForestClassifier: " )
    clf = RandomForestClassifier(n_estimators=50, random_state=1)
    timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)
    print(matriz_confusao)
    all_res.append(["RandomForestClassifier: ", timee , acuracia, precision, recall, f1_score])

    ### ExtraTreesClassifier
    print( "ExtraTreesClassifier: ")
    clf = ExtraTreesClassifier(n_estimators=100, random_state=0)
    timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)
    print(matriz_confusao)
    all_res.append(["ExtraTreesClassifier: ", timee , acuracia, precision, recall, f1_score])

    ### LogisticRegression
    print( "LogisticRegression: ")
    clf = LogisticRegression(random_state=0)
    timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)
    print(matriz_confusao)
    all_res.append(["LogisticRegression: ",  timee , acuracia, precision, recall, f1_score])

    ### svm
    print( "svm: ")
    clf = svm.SVC()
    timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)
    print(matriz_confusao)
    all_res.append(["svm: ",  timee , acuracia, precision, recall, f1_score])

    ### Stacking_scikit
    print( "Stacking_scikit: ")
    clf1 = RandomForestClassifier(n_estimators=50, random_state=1)
    clf2 = LogisticRegression(random_state=0)
    clf3 = SVC()
    clf4 = ExtraTreesClassifier(n_estimators=100, random_state=0)
    estimators = [('rf', clf1), ('lr', clf2), ('svc', clf3), ('etc', clf4)]
    ini = time.time()
    clf = StackingClassifier( estimators=estimators, final_estimator=LogisticRegression(), n_jobs=None)
    clf.fit(X_train_v, y_train)
    Y_Previsto_stacking = clf.predict(X_test_v)
    fim = time.time()

    timee = fim-ini
    acuracia = accuracy_score(y_test, Y_Previsto_stacking)
    precision = precision_score(y_test, Y_Previsto_stacking, average='weighted')
    matriz_confusao =  confusion_matrix(y_test, Y_Previsto_stacking)
    recall = recall_score(y_test, Y_Previsto_stacking, average='weighted')
    f1_score = metrics.f1_score(y_test, Y_Previsto_stacking, average='weighted')
    print("Time: " + str(timee))
    print("Acurácia: " + str(acuracia))
    print("Precision: " + str(precision))
    print("Recal: " + str(recall))
    print("f1_score: " + str(f1_score))
    print(matriz_confusao)
    all_res.append(["Stacking_scikit: ",  timee , acuracia, precision, recall, f1_score])

    ### XGBClassifier
    #print( "XGBClassifier: ")
    #clf = XGBClassifier(eval_metric='mlogloss' )
    #timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)
    #print(matriz_confusao)
    #all_res.append(["XGBClassifier: ",  timee , acuracia, precision, recall, f1_score])

    ### OneVsRestClassifier_RF
    print( "OneVsRest_RF: ")
    clf = RandomForestClassifier(n_estimators=50, random_state=1)
    ovr = OneVsRestClassifier(clf)
    ini = time.time()
    ovr.fit(X_train_v, y_train)
    Y_Previsto_OVR_RF = ovr.predict(X_test_v)
    fim = time.time()


    timee = fim-ini
    acuracia = accuracy_score(y_test, Y_Previsto_OVR_RF)
    precision = precision_score(y_test, Y_Previsto_OVR_RF, average='weighted')
    matriz_confusao =  confusion_matrix(y_test, Y_Previsto_OVR_RF)
    recall = recall_score(y_test, Y_Previsto_OVR_RF, average='weighted')
    f1_score = metrics.f1_score(y_test, Y_Previsto_OVR_RF, average='weighted')
    print("Time: " + str(timee))
    print("Acurácia: " + str(acuracia))
    print("Precision: " + str(precision))
    print("Recal: " + str(recall))
    print("f1_score: " + str(f1_score))
    print(matriz_confusao)
    all_res.append(["OvR_RF: ",  timee , acuracia, precision, recall, f1_score])

    return pd.DataFrame( all_res, columns=["ALGORITHM","TIME","ACCURACY","PRECISION","RECALL", "F1_SCORE"])

feat = ['Idade', 'Peso Kg', 'Cor da pele', 'ACTN3',
       'AGT', 'ECA', 'Score Força%', 'Score Resistência %',
       'Pré CK (U/L)', 'Pós CK (U/L)', 'Pré CK MB U/L', 'Pós CK MB U/L',
       'Pré Lactato (mmol/L)', 'Pós Lactato (mmol/L)', 'Pré LDH (U/L)',
       'Pós LDH (U/L)', 'Pré AST (U/L)', 'Pós AST (U/L)',
       'Pré Albumina (g/dL)', 'Pós Albumina (g/dL)', 'Pré TGP (g/dL)',
       'Pós TGP (g/dL)', 'Pré GGT U/L', 'Pós GGT U/L', 'Pré Ac. Úrico (mg/dL)',
       'Pós Ac. Úrico (mg/dL)', 'Pré TFG   (ml/min/1.73 m2)',
       'Pós TFG   (ml/min/1.73 m2)', 'Pré Ureia mg/dL', 'Pós Ureia mg/dL',
       'Pré creatinina mg/dL', 'Pós creatinina mg/dL',
       'Delta CREATININA mg/dL', 'Pré Fósforo mg/dL', 'Pós Fósforo mg/dL',
       'Pré Proteína Total (g/dL)', 'Pós Proteína Total (g/dL)',
       'Pré FE (ug/dL)', 'Pós FE (ug/dL)', 'Pré calcio mg/dL',
       'Pós calcio mg/dL', 'Pré Potássio mmol/L', 'Pós Potássio mmol/L',
       'Glicose pré', 'Glicose pós', 'Pré Sódio mmol/L', 'Pós Sódio mmol/L',
       'Pré Magnésio mg/dL', 'Pós Magnésio mg/dL', 'Pré Cloro mmol/L',
       'Pós Cloro mmol/L',
       'Osmolalidade sérica PRÉ BD4(mOsm/kg): 1,86(Na + K) + 1,15(Gli /18) + (Ureia/6) + 14\t\t\t',
       'Osmolalidade sérica PÓS BD4(mOsm/kg): 1,86(Na + K) + 1,15(Gli /18) + (Ureia/6) + 14\t\t\t',
       'Pré RCB 10^6/mm³', 'Pós RCB 10^6/mm³', 'Pré HGB g/dL', 'Pós HGB g/dL',
       'Pré HCT %', 'Pós HCT %', 'Pré MCV fL', 'Pós MCV fL', 'Pré MCH pg',
       'Pós MCH pg', 'Pré MCHC g/dL', 'Pós MCHC g/dL', 'Pré PLT 10³/mm³',
       'Pós PLT 10³/mm³', 'Pré RDW %', 'Pós RDW %', 'Pré RET %', 'Pós RET %',
       'Pré WBC 10³/mm³', 'Pós WBC 10³/mm³', 'Pré NEUT %', 'Pós NEUT %',
       'Pré LINF %', 'Pós LINF %', 'Pré MONO %', 'Pós MONO %', 'Pré EOS %',
       'Pós EOS %', 'Pré BASO %', 'Pós BASO %', 'massa magra', 'MCT KG', 'ESTATURA', 'vo2max estimado',
       'Handgrip     Dom.', 'Handgrip Cont.', '% gordura']

label_rabdomiolise = "Rabdomiólise"
label_lesao = "Lesão Renal Aguda"




In [67]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/'
file = 'dados.xlsx'
file2 = 'dados_completos.xlsx'
DATASET = pd.read_excel(path + file)
DATASET2 = pd.read_excel(path + file2)
DATASET

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Ano  Missão  Aluno Idade Peso Kg Cor da pele ACTN3  AGT  ECA   BDKRB2  \
0    2018   CIASC   3150    33      88           1    XX   MT   ID  (-9,-9)   
1    2018   CIASC   3155    26      74           1    RR   MT   II  (+9,-9)   
2    2018   CIASC   3156    26      76           1    RX   MM   ID  (+9,+9)   
3    2018   CIASC   3159    25      84           1    RR   TT   DD  (+9,-9)   
4    2018   CIASC   3165    33      82           1    RR   MT   II  (+9,-9)   
..    ...     ...    ...   ...     ...         ...   ...  ...  ...      ...   
462  2023  Manaus   3340    29     NaN           2   NaN  NaN  NaN      NaN   
463  2023  Manaus   3343    27     NaN           2   NaN  NaN  NaN      NaN   
464  2023  Manaus   3347    28     NaN           2   NaN  NaN  NaN      NaN   
465  2023  Manaus   3348    29     NaN           2   NaN  NaN  NaN      NaN   
466  2023  Manaus   3350    29     NaN           2   NaN  NaN  NaN      NaN   

     ... Pós BASO % Lesão Renal Aguda Rabdomiólise  massa magra MCT KG  \
0    ...        0.1               0.0        False        75.52   84.3   
1    ...        0.2               0.0         True        69.49   75.9   
2    ...        0.3               0.0         True        72.07   79.2   
3    ...        0.9               0.0         True        75.26   84.3   
4    ...        0.1               0.0         True        66.26   79.6   
..   ...        ...               ...          ...          ...    ...   
462  ...        NaN               0.0        False        76,40   82.8   
463  ...        NaN               0.0        False        65,80   75.1   
464  ...        NaN               0.0        False        74,27   82.7   
465  ...        NaN               0.0        False        70,18   80.5   
466  ...        NaN               0.0        False        75,16   85.8   

     ESTATURA  vo2max estimado  Handgrip     Dom. Handgrip Cont.  % gordura  
0       181.5            50.85               35.0           37.0      10.05  
1       171.0            51.98               42.0           43.0       6.08  
2       174.0            52.42               49.5           49.5       8.72  
3       178.0            50.68               60.0           57.0      10.61  
4       182.0            47.64               51.0           49.0      14.84  
..        ...              ...                ...            ...        ...  
462     182.0            53,54               56.0           59.0       7,73  
463     170.0                -               48.0           60.0      12,39  
464     179.5            50,56               58.0           52.0      10,20  
465     181.0            51,80               60.0           50.0      12,82  
466     178.0            53,41               55.0           56.0      12,40  

[467 rows x 96 columns]

In [68]:
DATASET2

Ano  Missão  Aluno  Idade  Peso Kg  Cor da pele ACTN3 AGT ECA   BDKRB2  \
0    2018   CIASC   3150     33     88.0            1    XX  MT  ID  (-9,-9)   
1    2018   CIASC   3155     26     74.0            1    RR  MT  II  (+9,-9)   
2    2018   CIASC   3156     26     76.0            1    RX  MM  ID  (+9,+9)   
3    2018   CIASC   3159     25     84.0            1    RR  TT  DD  (+9,-9)   
4    2018   CIASC   3165     33     82.0            1    RR  MT  II  (+9,-9)   
..    ...     ...    ...    ...      ...          ...   ...  ..  ..      ...   
271  2019   Natal   3235     28     87.0            1    XX  TT  DD  (+9,-9)   
272  2019  Avelar   3197     27     78.5            1    RR  MM  DD  (-9,-9)   
273  2019  Avelar   3200     25     85.1            1    XX  TT  ID  (-9,-9)   
274  2019  Avelar   3222     28     85.0            1    RR  MT  ID  (-9,-9)   
275  2019  Avelar   3235     28     81.0            1    XX  TT  DD  (+9,-9)   

     ...  Pós BASO %  Lesão Renal Aguda  Rabdomiólise  massa magra  MCT KG  \
0    ...         0.1              False         False        75.52   84.30   
1    ...         0.2              False          True        69.49   75.90   
2    ...         0.3              False          True        72.07   79.20   
3    ...         0.9              False          True        75.26   84.30   
4    ...         0.1              False          True        66.26   79.60   
..   ...         ...                ...           ...          ...     ...   
271  ...         3.1              False          True        74.20   83.60   
272  ...         0.2              False          True        73.39   85.65   
273  ...         0.4              False         False        72.78   80.00   
274  ...         0.4              False          True        53.70   62.35   
275  ...         1.1               True          True        74.20   83.60   

     ESTATURA  vo2max estimado  Handgrip     Dom.  Handgrip Cont.  % gordura  
0       181.5            50.85               35.0            37.0      10.05  
1       171.0            51.98               42.0            43.0       6.08  
2       174.0            52.42               49.5            49.5       8.72  
3       178.0            50.68               60.0            57.0      10.61  
4       182.0            47.64               51.0            49.0      14.84  
..        ...              ...                ...             ...        ...  
271     175.0            50.56               54.0            42.0       8.43  
272     177.0            51.46               48.0            52.0      10.92  
273     177.5             0.00               42.0            42.0       7.58  
274     169.0            52.58               44.0            38.0       9.73  
275     175.0            50.56               54.0            42.0       8.43  

[276 rows x 96 columns]

In [69]:
DATASET.columns

Index(['Ano', 'Missão', 'Aluno', 'Idade', 'Peso Kg', 'Cor da pele', 'ACTN3',
       'AGT', 'ECA', 'BDKRB2', 'Score Força%', 'Score Resistência %',
       'Pré CK (U/L)', 'Pós CK (U/L)', 'Pré CK MB U/L', 'Pós CK MB U/L',
       'Pré Lactato (mmol/L)', 'Pós Lactato (mmol/L)', 'Pré LDH (U/L)',
       'Pós LDH (U/L)', 'Pré AST (U/L)', 'Pós AST (U/L)',
       'Pré Albumina (g/dL)', 'Pós Albumina (g/dL)', 'Pré TGP (g/dL)',
       'Pós TGP (g/dL)', 'Pré GGT U/L', 'Pós GGT U/L', 'Pré Ac. Úrico (mg/dL)',
       'Pós Ac. Úrico (mg/dL)', 'Pré TFG   (ml/min/1.73 m2)',
       'Pós TFG   (ml/min/1.73 m2)', 'Pré Ureia mg/dL', 'Pós Ureia mg/dL',
       'Pré creatinina mg/dL', 'Pós creatinina mg/dL',
       'Delta CREATININA mg/dL', 'Pré Fósforo mg/dL', 'Pós Fósforo mg/dL',
       'Pré Proteína Total (g/dL)', 'Pós Proteína Total (g/dL)',
       'Pré FE (ug/dL)', 'Pós FE (ug/dL)', 'Pré calcio mg/dL',
       'Pós calcio mg/dL', 'Pré Potássio mmol/L', 'Pós Potássio mmol/L',
       'Glicose pré', 'Glicose 

### Teste 1 -  'Pré CK (U/L)', 'Pós CK (U/L)', "Rabdomiólise"

In [70]:
col_label_rabdo = ["Rabdomiólise"]
col_label_lesao = ["Lesão Renal Aguda"]
col_feature = ['Pré CK (U/L)', 'Pós CK (U/L)']
col_to_be_encoded = ["Rabdomiólise", "Lesão Renal Aguda"]


col_df = ['Pré CK (U/L)', 'Pós CK (U/L)', "Rabdomiólise", "Lesão Renal Aguda"]


df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()

for c in col_df:
    df = df.drop(df[df[c] == '-'].index)


for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])


for d in col_feature:
    df[d] = df[d].astype(float)

df

Pré CK (U/L)  Pós CK (U/L)  Rabdomiólise  Lesão Renal Aguda
0           337.0         664.0             0                  0
1           687.0        2246.0             1                  0
2           841.0        4616.0             1                  0
3           363.0        2611.0             1                  0
4           192.0        1111.0             1                  0
..            ...           ...           ...                ...
423         223.0        1570.0             1                  0
424         340.0       11324.0             1                  1
425         312.0        4688.0             1                  0
426         109.0        6710.0             1                  0
427          90.0        4504.0             1                  0

[390 rows x 4 columns]

Prevendo Rabdomiólise

In [71]:
X_train_rabdo, X_test_rabdo, y_train_rabdo, y_test_rabdo = train_test_split(
    df[col_feature],
    df["Rabdomiólise"],
    train_size=0.80,
    random_state=28
)

results_rabdo = get_tests_result(X_train_rabdo, X_test_rabdo, y_train_rabdo, y_test_rabdo)
print("Resultados para previsão de Rabdomiólise:\n\n")
results_rabdo

MLPClassifier
Time: 0.2513418197631836
Acurácia: 0.6538461538461539
Precision: 0.5975367915666423
Recal: 0.6538461538461539
f1_score: 0.6081554808827536
[[ 4 20]
 [ 7 47]]
AdaBoostClassifier: 
Time: 0.10719156265258789
Acurácia: 1.0
Precision: 1.0
Recal: 1.0
f1_score: 1.0
[[24  0]
 [ 0 54]]
Voting_LR3_SVC1_ETC2: 
Time: 0.31032776832580566
Acurácia: 0.9487179487179487
Precision: 0.9487179487179487
Recal: 0.9487179487179487
f1_score: 0.9487179487179487
[[22  2]
 [ 2 52]]
DecisionTreeClassifier: 
Time: 0.0038590431213378906
Acurácia: 1.0
Precision: 1.0
Recal: 1.0
f1_score: 1.0
[[24  0]
 [ 0 54]]
RandomForestClassifier: 
Time: 0.08725309371948242
Acurácia: 1.0
Precision: 1.0
Recal: 1.0
f1_score: 1.0
[[24  0]
 [ 0 54]]
ExtraTreesClassifier: 
Time: 0.13286066055297852
Acurácia: 0.9615384615384616
Precision: 0.9623222060957911
Recal: 0.9615384615384616
f1_score: 0.9617511994013997
[[23  1]
 [ 2 52]]
LogisticRegression: 
Time: 0.010554075241088867
Acurácia: 0.9487179487179487
Precision: 0.9487

ALGORITHM      TIME  ACCURACY  PRECISION    RECALL  F1_SCORE
0           MLPClassifier:   0.251342  0.653846   0.597537  0.653846  0.608155
1      AdaBoostClassifier:   0.107192  1.000000   1.000000  1.000000  1.000000
2    Voting_LR3_SVC1_ETC2:   0.310328  0.948718   0.948718  0.948718  0.948718
3  DecisionTreeClassifier:   0.003859  1.000000   1.000000  1.000000  1.000000
4  RandomForestClassifier:   0.087253  1.000000   1.000000  1.000000  1.000000
5    ExtraTreesClassifier:   0.132861  0.961538   0.962322  0.961538  0.961751
6      LogisticRegression:   0.010554  0.948718   0.948718  0.948718  0.948718
7                     svm:   0.004697  0.910256   0.930521  0.910256  0.912857
8         Stacking_scikit:   1.411352  0.987179   0.987692  0.987179  0.987250
9                  OvR_RF:   0.085592  1.000000   1.000000  1.000000  1.000000

Prevendo Lesão Renal Aguda

In [72]:
X_train_lesao, X_test_lesao, y_train_lesao, y_test_lesao = train_test_split(
    df[col_feature],
    df["Lesão Renal Aguda"],
    train_size=0.80,
    random_state=28
)

results_lesao = get_tests_result(X_train_lesao, X_test_lesao, y_train_lesao, y_test_lesao)
print("Resultados para previsão de Lesão Renal Aguda:\n\n")
results_lesao

MLPClassifier
Time: 0.04509568214416504
Acurácia: 0.8589743589743589
Precision: 0.7378369493754109
Recal: 0.8589743589743589
f1_score: 0.7938107869142351
[[67  0]
 [11  0]]
AdaBoostClassifier: 
Time: 0.105804443359375
Acurácia: 0.8589743589743589
Precision: 0.7378369493754109
Recal: 0.8589743589743589
f1_score: 0.7938107869142351
[[67  0]
 [11  0]]
Voting_LR3_SVC1_ETC2: 
Time: 0.3163783550262451
Acurácia: 0.8589743589743589
Precision: 0.7378369493754109
Recal: 0.8589743589743589
f1_score: 0.7938107869142351
[[67  0]
 [11  0]]
DecisionTreeClassifier: 
Time: 0.0045795440673828125
Acurácia: 0.717948717948718
Precision: 0.755067155067155
Recal: 0.717948717948718
f1_score: 0.7353057199211046
[[54 13]
 [ 9  2]]
RandomForestClassifier: 
Time: 0.08890914916992188
Acurácia: 0.8205128205128205
Precision: 0.8339250493096646
Recal: 0.8205128205128205
f1_score: 0.8266317016317015
[[59  8]
 [ 6  5]]
ExtraTreesClassifier: 
Time: 0.13056135177612305
Acurácia: 0.8205128205128205
Precision: 0.8205128205

ALGORITHM      TIME  ACCURACY  PRECISION    RECALL  F1_SCORE
0           MLPClassifier:   0.045096  0.858974   0.737837  0.858974  0.793811
1      AdaBoostClassifier:   0.105804  0.858974   0.737837  0.858974  0.793811
2    Voting_LR3_SVC1_ETC2:   0.316378  0.858974   0.737837  0.858974  0.793811
3  DecisionTreeClassifier:   0.004580  0.717949   0.755067  0.717949  0.735306
4  RandomForestClassifier:   0.088909  0.820513   0.833925  0.820513  0.826632
5    ExtraTreesClassifier:   0.130561  0.820513   0.820513  0.820513  0.820513
6      LogisticRegression:   0.008734  0.858974   0.737837  0.858974  0.793811
7                     svm:   0.005692  0.858974   0.737837  0.858974  0.793811
8         Stacking_scikit:   1.452204  0.858974   0.737837  0.858974  0.793811
9                  OvR_RF:   0.092216  0.820513   0.833925  0.820513  0.826632

Salvando os resultados

In [78]:
saving_results(results_rabdo, path + "ResultadosTestesPIBITI/Teste1/", "resultados_rabdomiolise.csv")

saving_results(results_lesao, path + "ResultadosTestesPIBITI/Teste1/", "resultados_lesao_renal_aguda.csv")

# Teste 2 - Pré CK MB e Pós CK MB

In [ ]:
col_label = ["Rabdomiólise"]
label_lesao = ["Lesão Renal Aguda"]
col_feature = ['Pré CK MB U/L', 'Pós CK MB U/L']
col_to_be_encoded = ["Rabdomiólise"]

col_df = ['Pré CK MB U/L', 'Pós CK MB U/L', "Rabdomiólise" ]

df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()
for c in col_df:
    df = df.drop(df[df[c] == '-'].index)

for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])

# convert to float
for d in col_feature:
    df[d] = df[d].astype(float)

df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( df[col_feature],
                                                    df[col_label],
                                                    train_size=0.80, random_state=28)

results = get_tests_result(X_train, X_test, y_train, y_test)
results

# Teste 3 - 'ACTN3',  'AGT', 'ECA', 'BDKRB2', 'Score Força%', 'Score Resistência %,

In [ ]:
col_label = ["Rabdomiólise"]
label_lesao = ["Lesão Renal Aguda"]
col_feature = ['ACTN3', 'AGT', 'ECA', 'BDKRB2', 'Score Força%', 'Score Resistência %']
col_to_be_encoded = ['ACTN3', 'AGT', 'ECA', 'BDKRB2', "Rabdomiólise"]
col_to_float = ['Score Força%', 'Score Resistência %']

col_df = ['ACTN3', 'AGT', 'ECA', 'BDKRB2', 'Score Força%', 'Score Resistência %', "Rabdomiólise" ]

df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()
for c in col_df:
    df = df.drop(df[df[c] == '-'].index)

for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])

# convert to float
for d in col_to_float:
    df[d] = df[d].astype(float)

df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( df[col_feature],
                                                    df[col_label],
                                                    train_size=0.80, random_state=28)

results = get_tests_result(X_train, X_test, y_train, y_test)
results

# Teste 4 - 'Idade', 'Peso Kg', 'Cor da pele',  'Score Força%', 'Score Resistência %'

In [ ]:
col_label = ["Rabdomiólise"]
label_lesao = ["Lesão Renal Aguda"]
col_feature = ['Idade', 'Peso Kg', 'Cor da pele', 'Score Força%', 'Score Resistência %']
col_to_be_encoded = ["Cor da pele", "Rabdomiólise"]
col_to_float = ['Idade', 'Peso Kg', 'Score Força%', 'Score Resistência %']

col_df = ['Idade', 'Peso Kg', 'Cor da pele', 'Score Força%', 'Score Resistência %', "Rabdomiólise" ]

df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()
for c in col_df:
    df = df.drop(df[df[c] == '-'].index)

for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])

# convert to float
for d in col_to_float:
    df[d] = df[d].astype(float)

df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( df[col_feature],
                                                    df[col_label],
                                                    train_size=0.80, random_state=28)

results = get_tests_result(X_train, X_test, y_train, y_test)
results

# Teste 5 - 'Pré Lactato (mmol/L)', 'Pós Lactato (mmol/L)'

In [ ]:
col_label = ["Rabdomiólise"]
label_lesao = ["Lesão Renal Aguda"]
col_feature = ['Pré Lactato (mmol/L)', 'Pós Lactato (mmol/L)']
col_to_be_encoded = ["Rabdomiólise"]
col_to_float = ['Pré Lactato (mmol/L)', 'Pós Lactato (mmol/L)']

col_df = ['Pré Lactato (mmol/L)', 'Pós Lactato (mmol/L)', "Rabdomiólise" ]

df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()
for c in col_df:
    df = df.drop(df[df[c] == '-'].index)

for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])

# convert to float
for d in col_to_float:
    df[d] = df[d].astype(float)

df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( df[col_feature],
                                                    df[col_label],
                                                    train_size=0.80, random_state=28)

results = get_tests_result(X_train, X_test, y_train, y_test)
results

# Teste 6 - 'Pré LDH (U/L)',  'Pós LDH (U/L)', 'Pré AST (U/L)', 'Pós AST (U/L)',
       'Pré Albumina (g/dL)', 'Pós Albumina (g/dL)', 'Pré TGP (g/dL)',
       'Pós TGP (g/dL)', 'Pré GGT U/L', 'Pós GGT U/L', 'Pré Ac. Úrico (mg/dL)',
       'Pós Ac. Úrico (mg/dL)', 'Pré TFG   (ml/min/1.73 m2)',
       'Pós TFG   (ml/min/1.73 m2)', 'Pré Ureia mg/dL', 'Pós Ureia mg/dL',
       'Pré creatinina mg/dL', 'Pós creatinina mg/dL',
       'Delta CREATININA mg/dL', 'Pré Fósforo mg/dL', 'Pós Fósforo mg/dL',
       'Pré Proteína Total (g/dL)', 'Pós Proteína Total (g/dL)',
       'Pré FE (ug/dL)', 'Pós FE (ug/dL)', 'Pré calcio mg/dL',
       'Pós calcio mg/dL', 'Pré Potássio mmol/L', 'Pós Potássio mmol/L',
       'Glicose pré', 'Glicose pós', 'Pré Sódio mmol/L', 'Pós Sódio mmol/L',
       'Pré Magnésio mg/dL', 'Pós Magnésio mg/dL', 'Pré Cloro mmol/L',
       'Pós Cloro mmol/L'

In [ ]:
col_label = ["Rabdomiólise"]
label_lesao = ["Lesão Renal Aguda"]
col_feature = ['Pré LDH (U/L)', 'Pós LDH (U/L)', 'Pré AST (U/L)', 'Pós AST (U/L)',
       'Pré Albumina (g/dL)', 'Pós Albumina (g/dL)', 'Pré TGP (g/dL)',
       'Pós TGP (g/dL)', 'Pré GGT U/L', 'Pós GGT U/L', 'Pré Ac. Úrico (mg/dL)',
       'Pós Ac. Úrico (mg/dL)', 'Pré TFG   (ml/min/1.73 m2)',
       'Pós TFG   (ml/min/1.73 m2)', 'Pré Ureia mg/dL', 'Pós Ureia mg/dL',
       'Pré creatinina mg/dL', 'Pós creatinina mg/dL',
       'Delta CREATININA mg/dL', 'Pré Fósforo mg/dL', 'Pós Fósforo mg/dL',
       'Pré Proteína Total (g/dL)', 'Pós Proteína Total (g/dL)',
       'Pré FE (ug/dL)', 'Pós FE (ug/dL)', 'Pré calcio mg/dL',
       'Pós calcio mg/dL', 'Pré Potássio mmol/L', 'Pós Potássio mmol/L',
       'Glicose pré', 'Glicose pós', 'Pré Sódio mmol/L', 'Pós Sódio mmol/L',
       'Pré Magnésio mg/dL', 'Pós Magnésio mg/dL', 'Pré Cloro mmol/L',
       'Pós Cloro mmol/L']
col_to_be_encoded = ["Rabdomiólise"]
col_to_float = ['Pré LDH (U/L)', 'Pós LDH (U/L)', 'Pré AST (U/L)', 'Pós AST (U/L)',
                'Pré Albumina (g/dL)',
                'Pós Albumina (g/dL)', 'Pré Ac. Úrico (mg/dL)',
       'Pós Ac. Úrico (mg/dL)', 'Pré TFG   (ml/min/1.73 m2)', 'Pós TFG   (ml/min/1.73 m2)',
                'Pré Ureia mg/dL', 'Pós Ureia mg/dL', 'Pré creatinina mg/dL', 'Pós creatinina mg/dL',
               'Delta CREATININA mg/dL', 'Pré Fósforo mg/dL', 'Pós Fósforo mg/dL',
                'Pré Proteína Total (g/dL)', 'Pós Proteína Total (g/dL)',
                'Pré FE (ug/dL)', 'Pós FE (ug/dL)', 'Pré calcio mg/dL',
       'Pós calcio mg/dL', 'Pré Potássio mmol/L', 'Pós Potássio mmol/L',
                'Pré Magnésio mg/dL', 'Pós Magnésio mg/dL', 'Pré Cloro mmol/L',
       'Pós Cloro mmol/L']

col_df = ['Pré LDH (U/L)', 'Pós LDH (U/L)', 'Pré AST (U/L)', 'Pós AST (U/L)',
       'Pré Albumina (g/dL)', 'Pós Albumina (g/dL)', 'Pré TGP (g/dL)',
       'Pós TGP (g/dL)', 'Pré GGT U/L', 'Pós GGT U/L', 'Pré Ac. Úrico (mg/dL)',
       'Pós Ac. Úrico (mg/dL)', 'Pré TFG   (ml/min/1.73 m2)',
       'Pós TFG   (ml/min/1.73 m2)', 'Pré Ureia mg/dL', 'Pós Ureia mg/dL',
       'Pré creatinina mg/dL', 'Pós creatinina mg/dL',
       'Delta CREATININA mg/dL', 'Pré Fósforo mg/dL', 'Pós Fósforo mg/dL',
       'Pré Proteína Total (g/dL)', 'Pós Proteína Total (g/dL)',
       'Pré FE (ug/dL)', 'Pós FE (ug/dL)', 'Pré calcio mg/dL',
       'Pós calcio mg/dL', 'Pré Potássio mmol/L', 'Pós Potássio mmol/L',
       'Glicose pré', 'Glicose pós', 'Pré Sódio mmol/L', 'Pós Sódio mmol/L',
       'Pré Magnésio mg/dL', 'Pós Magnésio mg/dL', 'Pré Cloro mmol/L',
       'Pós Cloro mmol/L', "Rabdomiólise" ]

df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()
for c in col_df:
    df = df.drop(df[df[c] == '-'].index)

for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])

# convert to float
for d in col_to_float:
    df[d] = df[d].astype(float)

df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( df[col_feature],
                                                    df[col_label],
                                                    train_size=0.80, random_state=28)

results = get_tests_result(X_train, X_test, y_train, y_test)
results

# Teste 7 - Série vermelha -  'Pré RCB 10^6/mm³', 'Pós RCB 10^6/mm³', 'Pré HGB g/dL', 'Pós HGB g/dL',
       'Pré HCT %', 'Pós HCT %', 'Pré MCV fL', 'Pós MCV fL', 'Pré MCH pg',
       'Pós MCH pg', 'Pré MCHC g/dL', 'Pós MCHC g/dL', 'Pré PLT 10³/mm³',
       'Pós PLT 10³/mm³', 'Pré RDW %', 'Pós RDW %', 'Pré RET %', 'Pós RET %'

In [ ]:
col_label = ["Rabdomiólise"]
label_lesao = ["Lesão Renal Aguda"]
col_feature = ['Pré RCB 10^6/mm³', 'Pós RCB 10^6/mm³', 'Pré HGB g/dL', 'Pós HGB g/dL',
       'Pré HCT %', 'Pós HCT %', 'Pré MCV fL', 'Pós MCV fL', 'Pré MCH pg',
       'Pós MCH pg', 'Pré MCHC g/dL', 'Pós MCHC g/dL', 'Pré PLT 10³/mm³',
       'Pós PLT 10³/mm³', 'Pré RDW %', 'Pós RDW %', 'Pré RET %', 'Pós RET %']
col_to_be_encoded = ["Rabdomiólise"]
col_to_float = ['Pré RCB 10^6/mm³', 'Pós RCB 10^6/mm³', 'Pré HGB g/dL', 'Pós HGB g/dL',
       'Pré HCT %', 'Pós HCT %', 'Pré MCV fL', 'Pós MCV fL', 'Pré MCH pg',
       'Pós MCH pg', 'Pré MCHC g/dL', 'Pós MCHC g/dL', 'Pré PLT 10³/mm³',
       'Pós PLT 10³/mm³', 'Pré RDW %', 'Pós RDW %', 'Pré RET %', 'Pós RET %']

col_df = ['Pré RCB 10^6/mm³', 'Pós RCB 10^6/mm³', 'Pré HGB g/dL', 'Pós HGB g/dL',
       'Pré HCT %', 'Pós HCT %', 'Pré MCV fL', 'Pós MCV fL', 'Pré MCH pg',
       'Pós MCH pg', 'Pré MCHC g/dL', 'Pós MCHC g/dL', 'Pré PLT 10³/mm³',
       'Pós PLT 10³/mm³', 'Pré RDW %', 'Pós RDW %', 'Pré RET %', 'Pós RET %', "Rabdomiólise" ]

df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()
for c in col_df:
    df = df.drop(df[df[c] == '-'].index)

for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])

# convert to float
for d in col_to_float:
    df[d] = df[d].astype(float)

df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( df[col_feature],
                                                    df[col_label],
                                                    train_size=0.80, random_state=28)

results = get_tests_result(X_train, X_test, y_train, y_test)
results

# Teste 8 - Série leucocitária - 'Pré WBC 10³/mm³', 'Pós WBC 10³/mm³', 'Pré NEUT %', 'Pós NEUT %',
       'Pré LINF %', 'Pós LINF %', 'Pré MONO %', 'Pós MONO %', 'Pré EOS %',
       'Pós EOS %', 'Pré BASO %', 'Pós BASO %'

In [ ]:
col_label = ["Rabdomiólise"]
label_lesao = ["Lesão Renal Aguda"]
col_feature = ['Pré WBC 10³/mm³', 'Pós WBC 10³/mm³', 'Pré NEUT %', 'Pós NEUT %',
       'Pré LINF %', 'Pós LINF %', 'Pré MONO %', 'Pós MONO %', 'Pré EOS %',
       'Pós EOS %', 'Pré BASO %', 'Pós BASO %']
col_to_be_encoded = ["Rabdomiólise"]
col_to_float = ['Pré WBC 10³/mm³', 'Pós WBC 10³/mm³', 'Pré NEUT %', 'Pós NEUT %',
       'Pré LINF %', 'Pós LINF %', 'Pré MONO %', 'Pós MONO %', 'Pré EOS %',
       'Pós EOS %', 'Pré BASO %', 'Pós BASO %']

col_df = ['Pré WBC 10³/mm³', 'Pós WBC 10³/mm³', 'Pré NEUT %', 'Pós NEUT %',
       'Pré LINF %', 'Pós LINF %', 'Pré MONO %', 'Pós MONO %', 'Pré EOS %',
       'Pós EOS %', 'Pré BASO %', 'Pós BASO %', "Rabdomiólise" ]

df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()
for c in col_df:
    df = df.drop(df[df[c] == '-'].index)

for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])

# convert to float
for d in col_to_float:
    df[d] = df[d].astype(float)

df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( df[col_feature],
                                                    df[col_label],
                                                    train_size=0.80, random_state=28)

results = get_tests_result(X_train, X_test, y_train, y_test)
results